In [1]:
%matplotlib notebook


# Create an experiment, simulate it and access results 

This notebook shows how to make a single simple experiment 

1. Loading a pre-existing model in Modelflow
2. Creating an experimet by updating some variables
3. Simulating the model
4. Visualizing the results 

This Notebook uses a  model for Pakistan described here: {cite:author}`Burns2021`

## Imports

Modelflow's modelclass includes most of the methods needed to manage a model in Modelflow.

In [2]:
from modelclass import model 
model.widescreen()
model.scroll_off()

In [3]:
%load_ext autoreload
%autoreload 2

## Load a pre-existing model, data and descriptions 
The file `pak.pcim` contains a dump of model equations, dataframe, simulation options and variable descriptions. The file has been created when onboarding the model. 
Examples can be found [here](../../onboard/eviews/onboard%20one%20model%20from%20%20wf1.html#onboard-a-model-defined-in-a-wf1-file)

In [5]:
mpak,baseline = model.modelload('../onboard/eviews/modelflowdumps/pak.pcim',alfa=0.7,run=1)

## Create an experiment
The variable ```mpak``` contains the model instance and the variable   ```baseline``` contains the pandas dataframe with all series. 

We want to make an experiment where these variables: PAKGGREVCO2CER PAKGGREVCO2GER PAKGGREVCO2OER are updated.

The  variables contains tax rates for CO2 emission from different sources. The descriptions can be retrieved like this: 

:::{margin} **Variable descriptions**
Are contained in the variable '''mpak.var_description''' 
which is a dictionary
:::

In [6]:
CTY="PAK"
mmodel = globals()[f'm{CTY.lower()}']
VARS_exp = "GGEXPCRNTCN GGEXPWAGECN GGREVGNFSCN"
VARS_rev = "GGREVDPITCN GGREVDCITCN"
VARS_out = "nygdpmktpkn ggexptotlcn ggrevtotlcn necongovtkn".upper()
VARS_gdp = 'nygdpmktpcn'.upper()


VARS_exp_names = ' '.join(f'{CTY}{v}' for v in VARS_exp.split())
print(VARS_exp_names)

VARS_rev_names = ' '.join(f'{CTY}{v}' for v in VARS_rev.split())
print(VARS_rev_names)

VARS_out_names = ' '.join(f'{CTY}{v}' for v in VARS_out.split())
print(VARS_out_names)

VARS_gdp_names = ' '.join(f'{CTY}{v}' for v in VARS_gdp.split())
print(VARS_gdp_names)

PAKGGEXPCRNTCN PAKGGEXPWAGECN PAKGGREVGNFSCN
PAKGGREVDPITCN PAKGGREVDCITCN
PAKNYGDPMKTPKN PAKGGEXPTOTLCN PAKGGREVTOTLCN PAKNECONGOVTKN
PAKNYGDPMKTPCN


In [7]:

l = VARS_exp_names.split()+VARS_rev_names.split()+VARS_out_names.split()+VARS_gdp_names.split()
for n in l: 
    varcomponent = f'{n:30}  {n[:3]} {n[3:5]} {n[5:8]} {n[8:12]} {n[12:13]} {n[13:14]}    {mmodel.var_description[n] if n in mmodel.allvar else "Not in model"}'
    print(varcomponent)
    

PAKGGEXPCRNTCN                  PAK GG EXP CRNT C N    Current Expenditures
PAKGGEXPWAGECN                  PAK GG EXP WAGE C N    Not in model
PAKGGREVGNFSCN                  PAK GG REV GNFS C N    Taxes on Goods and Services
PAKGGREVDPITCN                  PAK GG REV DPIT C N    Not in model
PAKGGREVDCITCN                  PAK GG REV DCIT C N    Not in model
PAKNYGDPMKTPKN                  PAK NY GDP MKTP K N    Real GDP
PAKGGEXPTOTLCN                  PAK GG EXP TOTL C N    General government total expenditure (millions lcu)
PAKGGREVTOTLCN                  PAK GG REV TOTL C N    General government revenues, total (millions lcu)
PAKNECONGOVTKN                  PAK NE CON GOVT K N    Gov. Cons real
PAKNYGDPMKTPCN                  PAK NY GDP MKTP C N    GDP, Market Prices, LCU mn


In [8]:
l = mmodel[VARS_exp_names.split()+VARS_rev_names.split()].frml

PAKGGEXPCRNTCN : FRML <> PAKGGEXPCRNTCN = PAKGGEXPGNFSCN+PAKGGEXPINTPCN+PAKGGEXPTRNSCN $
PAKGGREVGNFSCN : FRML <Z,EXO> PAKGGREVGNFSCN = ((PAKGGREVGNFSXN/100*(PAKNECONPRVTCN+PAKNECONGOVTCN))+5.34672642860449E-12 + PAKGGREVGNFSCN_A)* (1-PAKGGREVGNFSCN_D)+ PAKGGREVGNFSCN_X*PAKGGREVGNFSCN_D $


In [9]:
alternative_base = mpak.fix(baseline,'PAKGGREVGNFSCN',2023,2023)
print(f'{baseline.loc[2023,"PAKGGREVGNFSCN_X"]=}')
print(f'{alternative_base.loc[2023,"PAKGGREVGNFSCN_X"]=}')

baseline.loc[2023,"PAKGGREVGNFSCN_X"]=0.0
alternative_base.loc[2023,"PAKGGREVGNFSCN_X"]=2804746.561631753


In [10]:
alternative = alternative_base.upd(f'<2023 2023> PAKGGREVGNFSCN_X + {baseline.loc[2023,VARS_gdp_names]*0.01 }',lprint=1)

Update + [676046.5434129941] 2023 2023
PAKGGREVGNFSCN_X               Before                After                 Diff
2023                     2804746.5616         3480793.1050          676046.5434


In [11]:
print(f'{alternative_base.loc[2023,"PAKGGREVGNFSCN_X"]=}')
print(f'{alternative.loc[2023,"PAKGGREVGNFSCN_X"]=}')

alternative_base.loc[2023,"PAKGGREVGNFSCN_X"]=2804746.561631753
alternative.loc[2023,"PAKGGREVGNFSCN_X"]=3480793.1050447472


## Run the model with all the scenarios

In [12]:
result = mpak(alternative,2020,2100) # simulates the model 

## Access results 

Now we have two dataframes with results ```baseline``` and ```result```. These dataframes can be manipulated and visualized
with the tools provided by the **pandas** library and other like **Matplotlib** and **Plotly**. However to make things easy the first and
latest simulation result is also in the mpak object:

- **mpak.basedf**: Dataframe with the values for baseline
- **mpak.lastdf**: Dataframe with the values for alternative  



In [13]:
with mpak.set_smpl(2020,2030):
    display(mpak[VARS_out_names])

In [14]:
mmodel.modeldash('PAKGGEXPCRNTCN' , jupyter=1,dashport=5004)

apprun
Dash app running on http://127.0.0.1:5004/


Dash_graph(mmodel=<
Model name                              :                  PAK 
Model structure                         :         Simultaneous 
Number of variables                     :                  841 
Number of exogeneous  variables         :                  463 
Number of endogeneous variables         :                  378 
>, pre_var='PAKGGEXPCRNTCN', filter=0, up=1, down=0, time_att=False, attshow=False, all=False, dashport=5004, debug=False, jupyter=1, show_trigger=False, inline=False, lag=False, threshold=0.5, growthshow=False)